In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
all_transforms = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor(),
                                      transforms.Normalize(mean = [0.4914,0.4822,0.4465],
                                                          std=[0.2023,0.1994,0.2010])])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root = '\data',train = True,transform = all_transforms,download = True)

Files already downloaded and verified


In [5]:
test_dataset = torchvision.datasets.CIFAR10(root = '\data',train = False,download = True,transform = all_transforms)

Files already downloaded and verified


In [6]:
batch_size = 64
epochs = 20
lr = 0.001
num_classes = 10

In [7]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle = True)

In [8]:
test_loader =torch.utils.data.DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle = True)

In [9]:
class CNN(nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=3,out_channels = 32,kernel_size=3)
        self.cnn2 = nn.Conv2d(in_channels=32,out_channels = 32,kernel_size=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride = 2)
        
        self.cnn3 = nn.Conv2d(in_channels = 32,out_channels=64,kernel_size=3)
        self.cnn4 = nn.Conv2d(in_channels = 64,out_channels=64,kernel_size=3)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.fc1 = nn.Linear(1600,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128,num_classes)
        
    def forward(self,x):
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.maxpool1(x)
        x = self.cnn3(x)
        x = self.cnn4(x)
        x = self.maxpool2(x)
        x = x.reshape(x.size(0),-1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        return x

In [11]:
model = CNN(num_classes)
model.to(device)

CNN(
  (cnn1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (cnn4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [12]:
loss = nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.SGD(model.parameters(),lr = lr,weight_decay = 0.005,momentum = 0.9)

In [14]:
total_step = len(train_loader)

In [15]:
device

'cuda'

In [19]:
for epoch in range(epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images).cuda()
        l = loss(output,labels)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, l.item()))

            
            

Epoch [1/20], Loss: 1.2107
Epoch [2/20], Loss: 1.0993
Epoch [3/20], Loss: 1.2231
Epoch [4/20], Loss: 1.4987
Epoch [5/20], Loss: 1.4061
Epoch [6/20], Loss: 0.5872
Epoch [7/20], Loss: 1.0338
Epoch [8/20], Loss: 0.8155
Epoch [9/20], Loss: 0.8702
Epoch [10/20], Loss: 0.8161
Epoch [11/20], Loss: 0.6221
Epoch [12/20], Loss: 0.7173
Epoch [13/20], Loss: 0.9381
Epoch [14/20], Loss: 0.6037
Epoch [15/20], Loss: 0.4948
Epoch [16/20], Loss: 0.7875
Epoch [17/20], Loss: 0.5585
Epoch [18/20], Loss: 0.4966
Epoch [19/20], Loss: 0.7505
Epoch [20/20], Loss: 0.5378


In [20]:
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct+=(predicted==labels).sum().item()
        
    print('Accuracy of the network on the {} train images: {} % '.format(50000,100*correct/total))




Accuracy of the network on the 50000 train images: 85.816 % 
